### https://github.com/vyomshm/Skip-gram-Word2vec/blob/master/Skip-Gram%2Bword2vec.ipynb - TF

https://github.com/rahul1728jha/Word2Vec_Implementation/blob/master/Word_2_Vec.ipynb - Scratch MAIN

https://github.com/SeonbeomKim/TensorFlow-Word2vec/blob/master/1.%20Word2vec%20-%20skip-gram/skip-gram.py - TF
https://github.com/ujhuyz0110/wrd_emb/blob/master/word2vec_skipgram_medium_v1.ipynb - TF

https://github.com/svaderia/SkipGram - TF


https://github.com/n0obcoder/Skip-Gram-Model-PyTorch - PyTorch

In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        


In [3]:
with open('data/text8') as f:
    text = f.read()

In [6]:
words = utils.preprocess(text)
print(words[:30])
new_text = [" ".join(words[0:30])]

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


['anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst']

In [7]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


In [8]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

In [9]:
from collections import Counter
np.random.seed(123)

def sub_sampling(int_words, threshold):
    word_counter = Counter(int_words)
    total_count = len(int_words)
    freq= {word: count/total_count for word,count in word_counter.items()}
    p_drop ={word:(1-np.sqrt(threshold/ freq[word])) for word in word_counter}
    trained_words = [word for word in int_words if p_drop[word]<np.random.random()]
    return trained_words

train_words = sub_sampling(int_words, threshold= 1e-5)

In [10]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = set(words[start:idx] + words[idx+1:stop+1])
    
    return list(target_words)

In [ ]:

def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y